In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import necessary libraries

In [ ]:
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten
import kerastuner as kt

# Read and preprocessing data

**"img_name" column is dropped because we do not need it.**


In [ ]:
mydata= pd.read_csv("../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")
mydata.drop("img_name",axis=1, inplace= True)
mydata.head(10)

# Visualisation for analysing the data

### Gender data

In [ ]:
plt.figure(figsize=(16,8))
sns.set_palette("Set2")
sns.countplot(x ='gender', data = mydata) 

As you see, they are almost equal, so we do not need to preprocess for the data which is gender. On the chart, "0" is labelled for man and "1" is labelled for woman.

### Ethnicity data

In [ ]:
plt.figure(figsize=(16,8))
sns.set_palette("muted")
sns.countplot(x ='ethnicity', data = mydata) 

As you see, there are some problems. Ethnicity with label "0" is too much if we look other labelled ethnicity. Also, Ethnicity with label "4" is very little. We should preprocess the data.

In [ ]:
count_eth=mydata[mydata["ethnicity"]==4]["ethnicity"].count()
len_eth=len(mydata["ethnicity"])
print("There are {} labelled data for ethnicity 4 and length of ethnicity data is {}".format(count_eth,len_eth))

### Age data

In [ ]:
plt.figure(figsize=(24,12))
sns.set_palette("Paired")
sns.countplot(x ="age", data = mydata) 
plt.xticks(rotation=90)
plt.show()


As you see, there are also some problems for age data. "26" has too much example and "70" and beyond has too less. So, we should preprocess the data.

In [ ]:
count_age=mydata[mydata["age"]==26]["age"].count()
len_age=len(mydata["age"])
print("There are {} labelled data for age 26 and length of age data is {}".format(count_age,len_age))

## Data preprocessing

### Preprocessing gender data

In [ ]:
x_gender= mydata.iloc[:,-1]

In [ ]:
img_data_gender=[]

for i in x_gender:
    image=np.array(i.split(" "),dtype="float32")
    image=image.reshape(48,48,1)
    img_data_gender.append(image)
    
img_data_gender=np.array(img_data_gender)
img_data_gender/=255
img_data_gender= img_data_gender.reshape(img_data_gender.shape[0],48,48,1)


y_gender= np.array([i for i in mydata["gender"]],dtype="int")
y_gender= to_categorical(y_gender,2)

### Preprocessing ethnicity data

In [ ]:
x_ethniticy=[]
y_ethniticy=[]
j=[0]

for i in range(len(mydata["ethnicity"])):
    if mydata["ethnicity"][i]==0:
        if j[0]<5000:
            j[0]+=1
            x_ethniticy.append(mydata["pixels"][i])
            y_ethniticy.append(mydata["ethnicity"][i])
            
    elif mydata["ethnicity"][i]==4:
        continue;

    else:
        x_ethniticy.append(mydata["pixels"][i])
        y_ethniticy.append(mydata["ethnicity"][i])
        

In [ ]:
img_data_eth=[]

for i in x_ethniticy:
    image=np.array(i.split(" "),dtype="float32")
    image=image.reshape(48,48,1)
    img_data_eth.append(image)
    
img_data_eth=np.array(img_data_eth)
img_data_eth/=255
img_data_eth= img_data_eth.reshape(img_data_eth.shape[0],48,48,1)

y_ethniticy= np.array(y_ethniticy, dtype="int")
y_ethnicity= to_categorical(y_ethniticy)

In [ ]:
df_eth=pd.DataFrame(data=y_ethniticy, columns=["ethnicity"])
plt.figure(figsize=(16,8))
sns.set_palette("muted")
sns.countplot(x ='ethnicity', data = df_eth) 

### Preprocessing age data

In [ ]:
x_age=[]
y_age=[]
j=0
for i in range(len(mydata["age"])):
    if mydata["age"][i]==26:
        if j<1200:
            j+=1
            x_age.append(mydata["pixels"][i])
            y_age.append(mydata["age"][i])
    elif mydata["age"][i]>70:
        continue
    else:
        x_age.append(mydata["pixels"][i])
        y_age.append(mydata["age"][i])
        

In [ ]:
img_data_age=[]
for i in x_age:
    image=np.array(i.split(" "),dtype="float32")
    image=image.reshape(48,48,1)
    img_data_age.append(image)
    
img_data_age=np.array(img_data_age)
img_data_age/=255
img_data_age= img_data_age.reshape(img_data_age.shape[0],48,48,1)

y_age= np.array(y_age, dtype="int")

In [ ]:
df_age=pd.DataFrame(data=y_age, columns=["age"])
plt.figure(figsize=(24,12))
sns.set_palette("Paired")
sns.countplot(x ='age', data = df_age) 
plt.xticks(rotation=90)
plt.show()

## Show random images

In [ ]:
random_image=[random.randint(0,len(img_data_gender)) for i in range(10)]

w=10
h=10
fig=plt.figure(figsize=(12, 10))
columns = 4
rows = 5
for i in range(1, 10):
    image = img_data_gender[random_image[i]] 
    fig.add_subplot(rows, columns, i)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(image)
    plt.xlabel("Gender:"+str(mydata['gender'].iloc[random_image[i]])+" Eth:"+str(mydata['ethnicity'].iloc[random_image[i]])+" Age:"+str(mydata['age'].iloc[random_image[i]]))
plt.show()

## Split datas for training and testing

In [ ]:
xtrain_gender, xtest_gender, ytrain_gender, ytest_gender= train_test_split(img_data_gender, y_gender, test_size=0.33, random_state=0)
xtrain_eth, xtest_eth, ytrain_eth, ytest_eth= train_test_split(img_data_eth, y_ethnicity, test_size=0.20, random_state=38)
xtrain_age, xtest_age, ytrain_age, ytest_age= train_test_split(img_data_age, y_age, test_size=0.33, random_state=38)


### Callback for models

In [ ]:
stop_early= keras.callbacks.EarlyStopping(monitor="val_loss",patience=2)

## Define models

We use keras API and keras tuner.

In [ ]:
def model_builder_gender(hp):
    image_input= keras.Input(shape=(48,48,1))
    x= Conv2D(32, kernel_size=3, activation="relu")(image_input)
    x= Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x= MaxPooling2D(2)(x)
    x= Dropout(0.30)(x)
    hp_kernel= hp.Choice("kernel_size", values=[3,5])
    x= Conv2D(128, kernel_size=hp_kernel, activation="relu")(x)
    x= MaxPooling2D(2)(x)
    x= BatchNormalization()(x)
    x= Flatten()(x)
    x= Dense(units=128, activation= "relu")(x)
    image_output= Dense(2,activation="softmax")(x)
    model= keras.Model(image_input, image_output)
    hp_learning_rate= hp.Choice("learning_rate", values=[1e-2,1e-3]);
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"])
    return model

In [ ]:
def model_builder_ethnicity(hp):
    image_input= keras.Input(shape=(48,48,1))
    x= Conv2D(16, kernel_size=5, activation="relu")(image_input)
    x= Conv2D(32, kernel_size=5, activation="relu")(x)
    x= Conv2D(64, kernel_size=5, activation="relu")(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    x= Dropout(0.30)(x)
    x= Conv2D(96, kernel_size=3, activation="relu")(x)
    x= Conv2D(128, kernel_size=1, activation="relu")(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    x= BatchNormalization()(x)
    x= Flatten()(x)
    hp_units= hp.Choice("units", values=[128,256])
    x= Dense(units=hp_units, activation= "relu")(x)
    image_output= Dense(4,activation="softmax")(x)
    model= keras.Model(image_input, image_output)
    hp_models= hp.Choice("model_type", values=["rmsprop","adam"])
    if hp_models=="rmsprop":
        model.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-3),
                loss=keras.losses.CategoricalCrossentropy(),
                metrics=["accuracy"])
    else:
        model.compile(optimizer= keras.optimizers.RMSprop(learning_rate=1e-3),
                loss=keras.losses.CategoricalCrossentropy(),
                metrics=["accuracy"])
        
    return model

In [ ]:
def model_builder_age(hp):
    image_input= keras.Input(shape=(48,48,1))
    x= Conv2D(16, kernel_size=3, activation="relu")(image_input)
    x= Conv2D(32, kernel_size=3, activation="relu")(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    x= Dropout(0.30)(x)
    x= Conv2D(64, kernel_size=3, activation="relu")(x)
    x= Conv2D(96, kernel_size=3, activation="relu")(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    x= Dropout(0.30)(x)
    x= Conv2D(128, kernel_size=3, activation="relu")(x)
    x= Conv2D(160, kernel_size=3, activation="relu")(x)
    x= BatchNormalization()(x)
    x= Flatten()(x)
    x= Dense(units=256, activation= "relu")(x)
    image_output= Dense(1)(x)
    model= keras.Model(image_input, image_output)
    hp_losses= hp.Choice("loss_type", values=["mean_absolute_percentage_error","mean_absolute_error"])
    if hp_losses=="mean_absolute_error":
        model.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-3),
                loss= "mean_absolute_error",
                metrics=["mean_squared_error"])
    else:
        model.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-2),
                loss= "mean_squared_error",
                metrics=["mean_squared_error"])
        
    return model

## Define tuners for each models

In [ ]:
tuner_gender = kt.Hyperband(model_builder_gender,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=2,
                     directory='my_dir',
                     project_name='face_age_gender')

In [ ]:
tuner_eth = kt.Hyperband(model_builder_ethnicity,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3,
                     directory='my_dir',
                     project_name='face_age_ethnicity')

In [ ]:
tuner_age = kt.Hyperband(model_builder_age,
                     objective='val_loss',
                     max_epochs=5,
                     factor=3,
                     directory='my_dir',
                     project_name='face_age_age')

## Searching best hyperparameters for each models with keras tuner

In [ ]:
tuner_gender.search(xtrain_gender, ytrain_gender , epochs=5, validation_split=0.2, callbacks=[stop_early])
best_hps_gender=tuner_gender.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search for gender  is complete. The optimal kernel size for the third convolutional layer is {best_hps_gender.get('kernel_size')}.
the optimal learning rate for the optimizer is {best_hps_gender.get('learning_rate')}. """)

In [ ]:
tuner_eth.search(xtrain_eth, ytrain_eth , epochs=5, validation_split=0.2, callbacks=[stop_early])
best_hps_eth=tuner_eth.get_best_hyperparameters(num_trials=1)[0]

print(f""" The hyperparameter search for ethnicity is complete. The optimal units for the first dense layer is {best_hps_eth.get('units')}. 
The optimal optimizer is {best_hps_eth.get('model_type')}. """)


In [ ]:
tuner_age.search(xtrain_age, ytrain_age , epochs=5, validation_split=0.2, callbacks=[stop_early])
best_hps_age=tuner_age.get_best_hyperparameters(num_trials=1)[0]

print(f"""The hyperparameter search for age is complete. The best loss type is {best_hps_age.get('loss_type')}. """)


## Find optimal epoch number for each models

In [ ]:
model_gender= tuner_gender.hypermodel.build(best_hps_gender)
history_gender= model_gender.fit(xtrain_gender, ytrain_gender, epochs=10, validation_split= 0.2)

val_accuracy_per_epoch_gender=history_gender.history["val_accuracy"]
best_epoch_gender= val_accuracy_per_epoch_gender.index(max(val_accuracy_per_epoch_gender))+1

print('Best epoch for gender: %d' % (best_epoch_gender,))

In [ ]:
model_eth= tuner_eth.hypermodel.build(best_hps_eth)
history_eth= model_eth.fit(xtrain_eth, ytrain_eth, epochs=10, validation_split= 0.2)

val_accuracy_per_epoch_eth=history_eth.history["val_accuracy"]
best_epoch_eth= val_accuracy_per_epoch_eth.index(max(val_accuracy_per_epoch_eth))+1

print('Best epoch for ethnicity: %d' % (best_epoch_eth,))

In [ ]:
model_age= tuner_age.hypermodel.build(best_hps_age)
history_age= model_age.fit(xtrain_age, ytrain_age, epochs=10, validation_split= 0.2)

val_loss_per_epoch_age=history_age.history["val_loss"]
best_epoch_age= val_loss_per_epoch_age.index(max(val_loss_per_epoch_age))+1

print('Best epoch for age: %d' % (best_epoch_age,))

## Train and test models

In [ ]:
hypermodel_gender = tuner_gender.hypermodel.build(best_hps_gender)
hypermodel_gender.fit(xtest_gender, ytest_gender, epochs=best_epoch_gender)

In [ ]:
eval_result_gender = hypermodel_gender.evaluate(xtest_gender, ytest_gender)

In [ ]:
hypermodel_eth = tuner_eth.hypermodel.build(best_hps_eth)
hypermodel_eth.fit(xtest_eth, ytest_eth, epochs=best_epoch_eth)

In [ ]:
eval_result_eth = hypermodel_eth.evaluate(xtest_eth, ytest_eth)

In [ ]:
hypermodel_age = tuner_age.hypermodel.build(best_hps_age)
hypermodel_age.fit(xtest_age, ytest_age, epochs=best_epoch_age)

In [ ]:
eval_result_age = hypermodel_age.evaluate(xtest_age, ytest_age)

## Validation results

In [ ]:
print("[test loss gender, test accuracy gender]:", eval_result_gender)
print("[test loss for ethnicity, test accuracy for ethnicity]:", eval_result_eth)
print("[test loss for age, test accuracy for age]:", eval_result_age)

## Results

In [ ]:
print(history_gender.history.keys())
print(history_eth.history.keys())
print(history_age.history.keys())

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history_gender.history['accuracy'], label='accuracy')
plt.plot(history_gender.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history_gender.history['loss'])
plt.plot(history_gender.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss'], loc='lower left')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history_eth.history['accuracy'], label='accuracy')
plt.plot(history_eth.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history_eth.history['loss'])
plt.plot(history_eth.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss'], loc='lower left')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history_age.history['loss'], label='mse')
plt.plot(history_age.history['val_loss'], label = 'mean_squared_error')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.legend(loc='lower right')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history_age.history['mean_squared_error'], label='mse')
plt.plot(history_age.history['val_mean_squared_error'], label = 'mean_squared_error')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.legend(loc='lower right')

## Conclusion

**We achieved over 90 percent success. I could have created a single model with multiple outputs, but the information contained in the data is different. If we had created a single model for all outputs, we would probably have had a lower success. Besides, we have to do different preprocessing for each output in the data. I hope you will like it.
I am waiting for your suggestions.**

https://www.tensorflow.org/tutorials/keras/keras_tuner

https://keras.io/getting_started/

https://seaborn.pydata.org